#### **Gruppo 5.b**: Caponi Marco (matricola: 508773) - Ceneda Gianluca (matricola: 488257)

# ANALISI E REVISIONE DEL PROGETTO LARSPLITTING 2D 

## CLASSE REFACTORING: coordintervals


Variabili utili per testare il funzionamento


In [25]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using BenchmarkTools
using OrderedCollections
using Base.Threads
using Test
using DataStructures


## Versione iniziale di coordintervals

coordintervals crea un dizionario ordinato dove la chiave è l'intervallo su una coordinata, e come valore associato ha l'indice dell'intervallo corrispondente nel boundig box

In [2]:
function coordintervals(coord,bboxes)
	boxdict = OrderedDict{Array{Float64,1},Array{Int64,1}}()
	for (h,box) in enumerate(bboxes)
		key = box[coord,:]
		if haskey(boxdict,key) == false
			boxdict[key] = [h]
		else
			push!(boxdict[key], h)
		end
	end
	return boxdict
end

coordintervals (generic function with 1 method)

In [6]:
@time coordintervals(1, [[0.0 1.0; 0.0 0.0], [0.0 0.0; 0.0 0.1]])   #0.000014 seconds

  0.000014 seconds (16 allocations: 1.188 KiB)


OrderedDict{Vector{Float64}, Vector{Int64}} with 2 entries:
  [0.0, 1.0] => [1]
  [0.0, 0.0] => [2]

### controllo se la funzione è type unstable

In [ ]:
@code_warntye coordintervals(1, [[0.0 1.0; 0.0 0.0], [0.0 0.0; 0.0 0.1]])

La funzione **non** è type unstable in quanto ritorna una stringa.

Body: :OrderedDict{Array{Float64,1},Array{Int64,1}}

### versione modificata coordintervals

Questa funzione è molto semplice non sono serviti grandi interventi. La condizione dell'if è diventata !haskey(boxdict,key) da haskey(boxdict,key) == false

In [23]:
function coordintervalsMOD(coord,bboxes)
     boxdict = OrderedDict{Array{Float64,1},Array{Int64,1}}()
     l = length(bboxes)
@simd     for h=1:l
@views         key = bboxes[h][coord,:]
         if !haskey(boxdict,key)
             boxdict[key] = [h]
         else 
             push!(boxdict[key], h)
         end
     end
     return boxdict
 end
 

coordintervalsMOD (generic function with 1 method)

In [21]:
@time coordintervalsMOD(1, [[0.0 1.0; 0.0 0.0], [0.0 0.0; 0.0 0.1]])   0.000013 seconds

  0.000013 seconds (16 allocations: 1.188 KiB)


OrderedDict{Vector{Float64}, Vector{Int64}} with 2 entries:
  [0.0, 1.0] => [1]
  [0.0, 0.0] => [2]

### Benchmark della funzione iniziale e modificata

funzione iniziale:

In [30]:
@benchmark coordintervals(1, [[0.0 1.0; 0.0 0.0], [0.0 0.0; 0.0 0.1]])

BenchmarkTools.Trial: 10000 samples with 14 evaluations.
 Range (min … max):  958.143 ns … 470.038 μs  ┊ GC (min … max): 0.00% … 99.54%
 Time  (median):       1.025 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.250 μs ±   6.902 μs  ┊ GC (mean ± σ):  9.11% ±  1.72%

  ▆██▇▆▅▄▃▃▂▂▁▁                                                 ▂
  █████████████▇▇▆▇▇▇▆▃▅▄▄▃▆▅▅▃▅▅▁▄▅▆▆▆▄▆▇▇▇▇▇█▆▇▅▆▅▄▅▃▅▁▁▁▃▃▄▃ █
  958 ns        Histogram: log(frequency) by time        2.5 μs <

 Memory estimate: 1.19 KiB, allocs estimate: 16.

funzione modificata:

In [31]:
@benchmark coordintervalsMOD(1, [[0.0 1.0; 0.0 0.0], [0.0 0.0; 0.0 0.1]])

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.059 μs … 953.791 μs  ┊ GC (min … max): 0.00% … 99.63%
 Time  (median):     1.134 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.749 μs ±  13.529 μs  ┊ GC (mean ± σ):  8.24% ±  1.41%

  █▇▆▄▂▁         ▁▁▁▁                                         ▂
  ██████████▇▇▇█████████▆▇▅▅▆▅▄▄▅▅▅▆▆▅▅▄▄▃▄▅▄▁▃▃▄▅▄▅▄▅▃▅▅▄▃▄▄ █
  1.06 μs      Histogram: log(frequency) by time      4.91 μs <

 Memory estimate: 1.19 KiB, allocs estimate: 16.

## TEST

test effettuati sulla funzione ottimizzata

In [ ]:
@testset "coordintervals Tests" begin
	# 2x2x2 cuboidal grid for 1-, 2-, and 3-dim tests
	V,(VV,EV,FV,CV) = Lar.cuboidGrid([2,2,2],true)
	W,_ = Lar.apply(Lar.r(1,1,pi/6),(V,[VV,EV,FV,CV]))
			
		cellpoints = [ W[:,EV[k]]::Lar.Points for k=1:length(EV) ]
		bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
		@testset "Edge tests" begin # 
			@test typeof(Lar.coordintervals(1,bboxes)) == 
				OrderedDict{Array{Float64,1}, Array{Int64,1}}
			@test typeof(Lar.coordintervals(2,bboxes)) == 
				OrderedDict{Array{Float64,1}, Array{Int64,1}}
			@test typeof(Lar.coordintervals(3,bboxes)) == 
				OrderedDict{Array{Float64,1}, Array{Int64,1}}
		end
		cellpoints = [ W[:,FV[k]]::Lar.Points for k=1:length(FV) ]
		bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
		@testset "Face tests" begin # 
			@test typeof(Lar.coordintervals(1,bboxes)) == 
				OrderedDict{Array{Float64,1}, Array{Int64,1}}
			@test typeof(Lar.coordintervals(2,bboxes)) == 
				OrderedDict{Array{Float64,1}, Array{Int64,1}}
			@test typeof(Lar.coordintervals(3,bboxes)) == 
				OrderedDict{Array{Float64,1}, Array{Int64,1}}
		end
		cellpoints = [ W[:,CV[k]]::Lar.Points for k=1:length(CV) ]
		bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
		@testset "Cell tests" begin # 
			@test typeof(Lar.coordintervals(1,bboxes)) == 
				OrderedDict{Array{Float64,1}, Array{Int64,1}}
			@test typeof(Lar.coordintervals(2,bboxes)) == 
				OrderedDict{Array{Float64,1}, Array{Int64,1}}
			@test typeof(Lar.coordintervals(3,bboxes)) == 
				OrderedDict{Array{Float64,1}, Array{Int64,1}}
		end
	end

![test di coordintervals](https://github.com/MarcoCap13/LAR-SPLITTING-2D-5.b-/blob/main/docs/test/coordintervals_test.png?raw=true)